In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm, tree
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier
import xgboost
import copy

### Load Dataset

In [ ]:
dataset = pd.read_csv('nba_player_data_through_jan.csv')
dataset.set_index('Player')
dataset = dataset.drop(columns = 'Unnamed: 0')
dataset.head()

### Data Preprocessing

In [ ]:
std_scaler = StandardScaler()
dataset_guards = dataset[dataset['Position'] == 'G']
dataset_bigs = dataset[dataset['Position'] == 'F/C']
x = dataset[['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'GP', 'W%']]
y = dataset['All-Star']
x_guards = dataset_guards[['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'GP', 'W%']]
y_guards = dataset_guards['All-Star']
x_bigs = dataset_bigs[['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'GP', 'W%']]
y_bigs = dataset_bigs['All-Star']
#Years in which the average league ppg is >= 105
year_ranges = [str(year) for year in range(2016,2019)] + [str(year) for year in range(1980,1993)]
#year_ranges = [str(year) for year in range(2000,2019)]
dataset_guards_restricted = dataset_guards[dataset_guards['Player'].str.split(' ', expand=True)[2].str.split('-',expand=True)[0].isin(year_ranges)]
dataset_bigs_restricted = dataset_bigs[dataset_bigs['Player'].str.split(' ', expand=True)[2].str.split('-',expand=True)[0].isin(year_ranges)]
x_guards_restricted = dataset_guards_restricted[['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'GP', 'W%']]
y_guards_restricted = dataset_guards_restricted['All-Star']
x_bigs_restricted = dataset_bigs_restricted[['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'GP', 'W%']]
y_bigs_restricted = dataset_bigs_restricted['All-Star']

### Classification Algorithm Performance Functions

In [ ]:
def check_classification_algorithms(algos, class_names, x_train, x_test, y_train, y_test): 
    for classifier in algos: 
        model = classifier.fit(x_train, y_train)
        preds = model.predict(x_test)
        print(classifier)
        print(confusion_matrix(y_test, preds))
        print(classification_report(y_test, preds, target_names=class_names))
def check_classification_k_fold_cross_validation(kfoldtype, algos, class_names, x_data, y_data):
    X, Y = np.array(x_data), np.array(y_data)
    for classifier in algos:
        cv_total_preds = []
        cv_total_real = []
        std_pipeline = make_pipeline(StandardScaler(), classifier)
        for train_ind, test_ind in kfoldtype.split(X, Y): 
            x_tr, x_te = X[train_ind], X[test_ind]
            y_tr, y_te = Y[train_ind], Y[test_ind]
            std_pipeline.fit(x_tr, y_tr)
            preds = std_pipeline.predict(x_te)
            cv_total_real = np.append(cv_total_real,y_te)
            cv_total_preds = np.append(cv_total_preds, preds)
        print(classifier)
        print(confusion_matrix(cv_total_real, cv_total_preds))
        print(classification_report(cv_total_real, cv_total_preds, target_names=class_names))

In [ ]:
classifiers = []
classifiers.append(LogisticRegression())
classifiers.append(KNeighborsClassifier(15))
classifiers.append(tree.DecisionTreeClassifier())
classifiers.append(RandomForestClassifier())
classifiers.append(AdaBoostClassifier())
classifiers.append(svm.SVC())
classifiers.append(xgboost.XGBClassifier())
classifiers.append(GaussianNB())
kfold = StratifiedKFold(10, True, 42)

### One model for all players

In [ ]:
check_classification_k_fold_cross_validation(kfold, classifiers, ['Non-All-Star', 'All-Star'], x ,y)
#Random Forest Seems to have the best performance

### Different Models for Guards and Bigs

In [ ]:
check_classification_k_fold_cross_validation(kfold, classifiers, ['Non-All-Star', 'All-Star'], x_guards ,y_guards)
#SVC seems to be really good for guards 
check_classification_k_fold_cross_validation(kfold, classifiers, ['Non-All-Star', 'All-Star'], x_bigs ,y_bigs)

### Restricting Dataset Based on League Scoring - May More Accurately Reflect Modern Scoring Trends
Dataset only contains years where average PPG >= 105 in the 3 point era (https://www.basketball-reference.com/leagues/NBA_stats_per_game.html) - similar to 2019-2020 (110.6 PPG)

In [ ]:
check_classification_k_fold_cross_validation(kfold, classifiers, ['Non-All-Star', 'All-Star'], x_guards_restricted ,y_guards_restricted)
check_classification_k_fold_cross_validation(kfold, classifiers, ['Non-All-Star', 'All-Star'], x_bigs_restricted ,y_bigs_restricted)
guards_model = svm.SVC(C=1, gamma='scale', kernel='rbf')
bigs_model = svm.SVC(C=100, gamma=0.01, kernel='rbf')
check_classification_k_fold_cross_validation(kfold, [guards_model], ['Non-All-Star', 'All-Star'], x_guards_restricted ,y_guards_restricted)
check_classification_k_fold_cross_validation(kfold, [bigs_model], ['Non-All-Star', 'All-Star'], x_bigs_restricted ,y_bigs_restricted)

### Random Search CV To Tune Random Forest Classifier For General Model
Used as reference: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 200, num = 1)]
max_depth.append(None)
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [1, 2, 4, 10]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
r_forest = RandomForestClassifier()
tuned_r_forest = RandomizedSearchCV(estimator = r_forest, 
                               param_distributions = random_grid, scoring='f1', n_iter = 100, cv = 10, 
                               verbose=2, random_state=42, n_jobs = -1)
tuner_r_forest.fit(x,y)
print(tuned_r_forest.best_params_)

### Grid Search CV for seperate guard/big models - Support Vector Classifier

In [ ]:
grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [2, 1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf', 'linear']} 
best_svc_guards = GridSearchCV(estimator=svm.SVC(), param_grid=grid, scoring='f1', cv=10, verbose = 3, n_jobs=-1) 
best_svc_bigs = GridSearchCV(estimator=svm.SVC(), param_grid=grid, scoring='f1', cv=10, verbose = 3, n_jobs=-1) 
best_svc_guards.fit(x_guards, y_guards)
best_svc_bigs.fit(x_bigs, y_bigs)

In [ ]:
print(best_svc_guards.best_estimator_)
print(best_svc_bigs.best_estimator_)
check_classification_k_fold_cross_validation(kfold, [best_svc_guards.best_estimator_], ['Non-All-Star', 'All-Star'], x_guards ,y_guards)
check_classification_k_fold_cross_validation(kfold, [best_svc_bigs.best_estimator_], ['Non-All-Star', 'All-Star'], x_bigs ,y_bigs)

### Comparing Tuned Random Forest General Model to Dummy Classifiers

In [ ]:
rforestcomp = []
rforestcomp.append(DummyClassifier(strategy='constant', constant=0))
rforestcomp.append(DummyClassifier('stratified'))
rforestcomp.append(DummyClassifier('prior'))
rforestcomp.append(DummyClassifier('uniform'))                     
check_classification_k_fold_cross_validation(kfold, rforestcomp, ['Non-All-Star', 'All-Star'], x ,y)

### Generate Predictions - Ultimately Using Support Vector Classifier Fitted on Restricted Dataset

In [ ]:
dataset_current = pd.read_csv('nba_player_data_through_jan_2019_2020.csv')
dataset_current.set_index('Player')
dataset_current_guards = dataset_current[dataset_current['Position'] == 'G']
dataset_current_bigs = dataset_current[dataset_current['Position'] == 'F/C']
x_current_guards = dataset_current_guards[['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'GP', 'W%']]
x_current_bigs = dataset_current_bigs[['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'GP', 'W%']]
guards_model = svm.SVC(C=1, gamma='scale', kernel='rbf')
bigs_model = svm.SVC(C=100, gamma=0.01, kernel='rbf')
guards_model.fit(x_guards_restricted, y_guards_restricted)
bigs_model.fit(x_bigs_restricted, y_bigs_restricted)
dataset_current_guards['All-Star'] = guards_model.predict(x_current_guards)
dataset_current_guards = dataset_current_guards[dataset_current_guards['All-Star']]
dataset_current_guards.to_csv('nba_all_star_predictions_guards_2019_2020.csv')
dataset_current_bigs['All-Star'] = bigs_model.predict(x_current_bigs)
dataset_current_bigs = dataset_current_bigs[dataset_current_bigs['All-Star']]
dataset_current_bigs.to_csv('nba_all_star_predictions_bigs_2019_2020.csv')